<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250618_Lab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 鐵達尼號乘客資料預處理與模型準備

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

class AutoPreprocess:
    def __init__(self):
        self.scaler = {}
        self.fillna_value = {}
        self.onehotencode_value = {}
        self.columns_order = None # 用於儲存訓練時的欄位順序

    def fit(self, df, numeric_cols=None, category_cols=None, fillna_strategy='mean'):
        """
        學習資料的預處理參數。

        Args:
            df (pd.DataFrame): 訓練資料。
            numeric_cols (list): 需要標準化的數值型欄位。
            category_cols (list): 需要進行獨熱編碼的類別型欄位。
            fillna_strategy (str): 數值型欄位的缺失值填充策略 ('mean' 或 'median')。
        """
        self.columns_order = df.columns.tolist() # 儲存原始欄位順序

        # 處理數值型欄位
        if numeric_cols:
            for col in numeric_cols:
                if col in df.columns:
                    if fillna_strategy == 'mean':
                        self.fillna_value[col] = df[col].mean()
                    elif fillna_strategy == 'median':
                        self.fillna_value[col] = df[col].median()
                    else:
                        self.fillna_value[col] = 0 # 預設填充為0，你也可以根據需求調整

                    # 填充缺失值後再擬合Scaler
                    temp_col = df[col].fillna(self.fillna_value[col]).values.reshape(-1, 1)
                    scaler = StandardScaler() if col == '船票價格' else MinMaxScaler() # '船票價格' 使用 StandardScaler，其他數值欄位使用 MinMaxScaler
                    scaler.fit(temp_col)
                    self.scaler[col] = scaler
                else:
                    print(f"Warning: Numeric column '{col}' not found in DataFrame.")

        # 處理類別型欄位
        if category_cols:
            for col in category_cols:
                if col in df.columns:
                    # 獨熱編碼會自動處理缺失值為一個新的類別
                    # 我們只需要記錄訓練集中的所有類別，以確保測試集有一致的欄位
                    self.onehotencode_value[col] = df[col].astype(str).unique().tolist()
                else:
                    print(f"Warning: Category column '{col}' not found in DataFrame.")


    def transform(self, df):
        """
        根據已學習的參數轉換資料。

        Args:
            df (pd.DataFrame): 需要轉換的資料。

        Returns:
            pd.DataFrame: 轉換後的資料。
        """
        transformed_df = df.copy()

        # 填充缺失值並標準化數值型欄位
        for col, scaler in self.scaler.items():
            if col in transformed_df.columns:
                if col in self.fillna_value:
                    transformed_df[col] = transformed_df[col].fillna(self.fillna_value[col])
                transformed_df[col] = scaler.transform(transformed_df[col].values.reshape(-1, 1))
            else:
                print(f"Warning: Numeric column '{col}' not found in DataFrame during transform. Skipping scaling.")


        # 獨熱編碼類別型欄位
        for col, categories in self.onehotencode_value.items():
            if col in transformed_df.columns:
                # 將新類別轉換為字符串，確保與訓練集類別類型一致
                transformed_df[col] = transformed_df[col].astype(str)
                # 對於測試集中可能出現的訓練集中不存在的類別，使用 handle_unknown='ignore'
                # 或者，更穩健的做法是確保所有類別都存在，這裡我們直接用 pd.get_dummies
                # 並且在訓練時記錄所有類別，在轉換時將其傳入
                dummy_cols = pd.get_dummies(transformed_df[col], prefix=col)
                # 確保轉換後的 df 包含訓練時的類別所有列，如果某些類別在轉換集中不存在，則補0
                for cat in categories:
                    dummy_col_name = f"{col}_{cat}"
                    if dummy_col_name not in dummy_cols.columns:
                        dummy_cols[dummy_col_name] = 0
                transformed_df = pd.concat([transformed_df.drop(columns=[col]), dummy_cols], axis=1)
            else:
                print(f"Warning: Category column '{col}' not found in DataFrame during transform. Skipping one-hot encoding.")


        # 重新排序欄位以匹配訓練時的順序
        # 這一步很重要，因為模型的特徵順序必須一致
        if self.columns_order:
            # 獲取原始數據框中非獨熱編碼的欄位
            original_non_onehot_cols = [col for col in self.columns_order if col not in self.onehotencode_value]

            # 獲取獨熱編碼後的欄位名稱
            onehot_encoded_cols = []
            for col in self.onehotencode_value:
                for category in self.onehotencode_value[col]:
                    onehot_encoded_cols.append(f"{col}_{category}")

            # 合併所有預期的欄位順序，並確保它們都在 transformed_df 中
            final_columns_order = []
            for col in original_non_onehot_cols:
                if col in transformed_df.columns:
                    final_columns_order.append(col)
            for col in onehot_encoded_cols:
                if col in transformed_df.columns:
                    final_columns_order.append(col)
                else:
                    # 如果訓練時存在的獨熱編碼欄位在轉換時不存在，則添加並填充0
                    transformed_df[col] = 0
                    final_columns_order.append(col) # 確保它在排序中

            # 過濾掉那些原始數據框中不存在的欄位，例如 '姓名', '船票編號', '船艙號碼'
            final_columns_order = [col for col in final_columns_order if col in transformed_df.columns]

            # 確保 '是否倖存' 列（如果存在）在轉換過程中不被排序，或者根據需求處理
            if '是否倖存' in transformed_df.columns and '是否倖存' not in final_columns_order:
                # 假設 '是否倖存' 是目標變數，通常不會對它進行預處理
                # 我們可以把它暫時移除，排序其他特徵，然後再加回來
                survived_col = transformed_df['是否倖存']
                transformed_df = transformed_df.drop(columns=['是否倖存'])
                transformed_df = transformed_df[final_columns_order]
                transformed_df['是否倖存'] = survived_col
            else:
                transformed_df = transformed_df[final_columns_order]


        return transformed_df

    def save(self, filepath):
        """儲存預處理器狀態"""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, filepath):
        """載入預處理器狀態"""
        with open(filepath, 'rb') as f:
            return pickle.load(f)

# --- 開始鐵達尼號資料預處理 ---

# 1. 讀取資料
print("正在讀取鐵達尼號訓練資料...")
try:
    df_train = pd.read_csv('Titanic(鐵達尼號乘客)_Train.csv')
    print("資料讀取成功！")
    print("資料前5行：")
    print(df_train.head())
    print("\n資料資訊：")
    df_train.info()
except FileNotFoundError:
    print("錯誤：找不到 'Titanic(鐵達尼號乘客)_Train.csv' 檔案。請確認檔案路徑是否正確。")
    # 如果找不到檔案，停止執行後續程式碼
    # exit() # 在Colab Notebook中不建議使用exit()，會停止整個Notebook的運行

# 2. 定義特徵和目標變數
# '是否倖存' 是我們的目標變數
target = '是否倖存'
# 移除不需要的欄位：'是否倖存', '乘客序號', '姓名', '船票編號', '船艙號碼'
features_to_drop = [target, '乘客序號', '姓名', '船票編號', '船艙號碼']
# 過濾出實際存在於df_train.columns中的欄位進行drop
existing_features_to_drop = [col for col in features_to_drop if col in df_train.columns]
features = df_train.drop(columns=existing_features_to_drop).columns.tolist()


# 根據鐵達尼號資料的特性，定義數值型和類別型欄位
# '年紀' 和 '船票價格' 是數值型，需要填充缺失值和標準化
# '船票等級', '性別', '旁系親屬數目', '直系親屬數目', '出發港口' 是類別型，需要獨熱編碼
numeric_cols = ['年紀', '船票價格']
category_cols = ['船票等級', '性別', '旁系親屬數目', '直系親屬數目', '出發港口']

# 確保所有選擇的欄位都存在於資料中
numeric_cols = [col for col in numeric_cols if col in features]
category_cols = [col for col in category_cols if col in features]

print(f"\n將進行預處理的數值型欄位：{numeric_cols}")
print(f"將進行預處理的類別型欄位：{category_cols}")

# 3. 使用 AutoPreprocess 進行預處理
print("\n正在初始化並擬合 AutoPreprocess 預處理器...")
auto_preprocess = AutoPreprocess()

# 將目標變數從特徵中移除，因為預處理器只處理特徵
# 這裡我們已經在前面drop的時候移除了target，所以X_train就是features
X_train = df_train[features]
y_train = df_train[target] # 目標變數通常不進行預處理，直接用於模型訓練

# 擬合預處理器
auto_preprocess.fit(X_train, numeric_cols=numeric_cols, category_cols=category_cols, fillna_strategy='mean')
print("預處理器擬合完成。")

# 轉換訓練資料
print("正在轉換訓練資料...")
X_train_processed = auto_preprocess.transform(X_train)
print("訓練資料轉換完成。")
print("\n轉換後資料前5行：")
print(X_train_processed.head())
print("\n轉換後資料資訊：")
X_train_processed.info()

# 4. 模型訓練（下一步）
print("\n--- 預處理完成，接下來是模型訓練步驟 ---")
print("在資料預處理完成後，下一步通常是：")
print("1. 選擇一個適合的機器學習模型（例如：邏輯迴歸、決策樹、隨機森林等）。")
print("2. 使用 `X_train_processed` 作為特徵，`y_train` 作為目標變數來訓練模型。")
print("3. 評估模型的性能。")
print("4. 如果有測試資料，使用相同的 `auto_preprocess` 實例來轉換測試資料，然後用訓練好的模型進行預測。")

# 範例：儲存預處理器以供未來使用
preprocess_filepath = 'titanic_auto_preprocess.pkl'
auto_preprocess.save(preprocess_filepath)
print(f"\n預處理器已儲存至：{preprocess_filepath}")

# 範例：載入預處理器
# loaded_preprocess = AutoPreprocess.load(preprocess_filepath)
# print(f"預處理器已從 {preprocess_filepath} 載入。")

正在讀取鐵達尼號訓練資料...
資料讀取成功！
資料前5行：
   乘客序號  船票等級                                                 姓名      性別  \
0     1     3                            Braund, Mr. Owen Harris    male   
1     2     1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2     3     3                             Heikkinen, Miss. Laina  female   
3     4     1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4     5     3                           Allen, Mr. William Henry    male   

     年紀  旁系親屬數目  直系親屬數目              船票編號     船票價格  船艙號碼 出發港口  是否倖存  
0  22.0       1       0         A/5 21171   7.2500   NaN    S     0  
1  38.0       1       0          PC 17599  71.2833   C85    C     1  
2  26.0       0       0  STON/O2. 3101282   7.9250   NaN    S     1  
3  35.0       1       0            113803  53.1000  C123    S     1  
4  35.0       0       0            373450   8.0500   NaN    S     0  

資料資訊：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns 

In [6]:
# --- 接續前一個程式碼的執行 ---

# 導入必要的機器學習函式庫
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

print("\n--- 開始模型訓練階段 ---")

# 1. 訓練/測試資料分割
# 將預處理後的訓練資料 X_train_processed 和目標變數 y_train 分割成訓練集和驗證集。
# test_size=0.2 表示 20% 的資料用於驗證，80% 用於訓練。
# random_state 確保每次運行時分割的結果都相同，以便於重現。
print("正在將資料分割為訓練集和驗證集 (80/20)...")
X_train_model, X_val_model, y_train_model, y_val_model = train_test_split(
    X_train_processed, y_train, test_size=0.2, random_state=42
)

print(f"訓練集特徵形狀: {X_train_model.shape}")
print(f"驗證集特徵形狀: {X_val_model.shape}")
print(f"訓練集目標形狀: {y_train_model.shape}")
print(f"驗證集目標形狀: {y_val_model.shape}")

# 2. 選擇和訓練模型
# 使用邏輯迴歸模型。
# solver='liblinear' 是一個常用的優化算法，對於小型數據集效果良好。
# random_state 確保模型訓練過程的隨機性部分可重現。
print("\n正在初始化並訓練邏輯迴歸模型...")
# model = LogisticRegression(solver='liblinear', random_state=42)
model = RandomForestClassifier()

model.fit(X_train_model, y_train_model)
print("模型訓練完成。")

# 3. 模型評估
# 使用驗證集進行預測。
print("\n正在使用驗證集評估模型性能...")
y_pred = model.predict(X_val_model)

# 計算準確度
accuracy = accuracy_score(y_val_model, y_pred)
print(f"模型在驗證集上的準確度: {accuracy:.4f}")

print("\n--- 模型訓練和評估完成 ---")
print("你可以嘗試不同的模型或調整模型參數，以尋找更好的性能。")
print("例如：隨機森林、梯度提升樹等。")

# 範例：儲存訓練好的模型
model_filepath = 'model.pkl'
with open(model_filepath, 'wb') as f:
    pickle.dump(model, f)
print(f"\n訓練好的模型已儲存至：{model_filepath}")

# 範例：載入訓練好的模型
# loaded_model = None
# with open(model_filepath, 'rb') as f:
#     loaded_model = pickle.load(f)
# print(f"模型已從 {model_filepath} 載入。")

# # 載入後的模型可以再次用於預測
# # example_prediction = loaded_model.predict(some_new_processed_data)


--- 開始模型訓練階段 ---
正在將資料分割為訓練集和驗證集 (80/20)...
訓練集特徵形狀: (712, 25)
驗證集特徵形狀: (179, 25)
訓練集目標形狀: (712,)
驗證集目標形狀: (179,)

正在初始化並訓練邏輯迴歸模型...
模型訓練完成。

正在使用驗證集評估模型性能...
模型在驗證集上的準確度: 0.8045

--- 模型訓練和評估完成 ---
你可以嘗試不同的模型或調整模型參數，以尋找更好的性能。
例如：隨機森林、梯度提升樹等。

訓練好的模型已儲存至：model.pkl
